In [1]:
import os
import numpy as np

from pyFM.mesh import TriMesh
from tqdm.auto import tqdm

import meshplot as mp

def plot_mesh(myMesh,cmap=None):
    mp.plot(myMesh.vertlist, myMesh.facelist,c=cmap)
    
def double_plot(myMesh1,myMesh2,cmap1=None,cmap2=None):
    d = mp.subplot(myMesh1.vertlist, myMesh1.facelist, c=cmap1, s=[2, 2, 0])
    mp.subplot(myMesh2.vertlist, myMesh2.facelist, c=cmap2, s=[2, 2, 1], data=d)

def visu(vertices):
    min_coord,max_coord = np.min(vertices,axis=0,keepdims=True),np.max(vertices,axis=0,keepdims=True)
    cmap = (vertices-min_coord)/(max_coord-min_coord)
    return cmap

In [2]:
meshlist = [TriMesh(f'data/shapes/shapes/{i:02d}.off', area_normalize=True, center=True).process(k=150, intrinsic=True) for i in range(381,400,1)]

In [3]:
map_files = os.listdir('data/bim_maps/bim_maps')

for idx, map_filename in tqdm(map_files):
    ind1 = idx
    
    for i in range(len(meshlist)):
        ind2 = i
        if ind2 == ind1:
            continue
        else:

            # Indicing starts at 1 in the names, but at 0 on the meshlist
            mesh1, mesh2 = meshlist[ind1], meshlist[ind2]
            
            # Load the pointwise map
            p2p_21 = np.loadtxt(f'../data/camel_gallop/maps/{map_filename}', dtype=int)

            if ind1 < 1 and ind2 < 4:
                cmap1 = visu(mesh1.vertlist); cmap2 = cmap1[p2p_21]
                double_plot(mesh1, mesh2, cmap1, cmap2)

  0%|          | 0/380 [00:00<?, ?it/s]

ValueError: too many values to unpack (expected 2)